Python: アンサンブル学習の Voting を試す
https://blog.amedama.jp/entry/2018/12/16/134028

今回は機械学習におけるアンサンブル学習の一種として Voting という手法を試してみる。 これは、複数の学習済みモデルを用意して多数決などで推論の結果を決めるという手法。 この手法を用いることで最終的なモデルの性能を上げられる可能性がある。 実装については自分で書いても良いけど scikit-learn に使いやすいものがあったので、それを選んだ。



---

以下のサンプルコードでは乳がんデータセットを使って Voting を試している。 使ったモデルはサポートベクターマシン、ランダムフォレスト、ロジスティック回帰、k-最近傍法、ナイーブベイズの五つ。 モデルの性能は 5-Fold CV を使って精度 (Accuracy) について評価している。

In [1]:
from collections import defaultdict

import numpy as np
from tqdm import tqdm
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB

def main():
    # 乳がんデータセットを読み込む
    dataset = datasets.load_breast_cancer()
    X, y = dataset.data, dataset.target

    # voting に使う分類器を用意する
    estimators = [
        ('svm', SVC(gamma='scale', probability=True)),
        ('rf', RandomForestClassifier(n_estimators=100)),
        ('logit', LogisticRegression(solver='lbfgs', max_iter=10000)),
        ('knn', KNeighborsClassifier()),
        ('nb', GaussianNB()),
    ]

    accs = defaultdict(list)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, test_index in tqdm(list(skf.split(X, y))):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # 分類器を学習する
        voting = VotingClassifier(estimators)
        voting.fit(X_train, y_train)

        # アンサンブルで推論する
        y_pred = voting.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accs['voting'].append(acc)

        # 個別の分類器の性能も確認してみる
        for name, estimator in voting.named_estimators_.items():
            y_pred = estimator.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            accs[name].append(acc)

    for name, acc_list in accs.items():
        mean_acc = np.array(acc_list).mean()
        print(name, ':', mean_acc)


if __name__ == '__main__':
    main()

100%|██████████| 5/5 [00:17<00:00,  3.57s/it]

voting : 0.9543238627542306
svm : 0.9138953578636858
rf : 0.9578326346840551
logit : 0.9543393882937432
knn : 0.9350100916006833
nb : 0.9385343890700202


なんと Voting するよりもランダムフォレスト単体の方が性能が良いという結果になってしまった。 このように Voting するからといって必ずしも性能が上がるとは限らない。 例えば今回のように性能が突出したモデルがあるなら、それ単体で使った方が良くなる可能性はある。 あるいは、極端に性能が劣るモデルがあるならそれは取り除いた方が良いかもしれない。 それ以外には、次の項目で説明するモデルの重み付けという手もありそう。

モデルに重みをつける


---


性能が突出したモデルを単体で使ったり、あるいは劣るモデルを取り除く以外の選択肢として、モデルの重み付けがある。 これは、多数決などで推論結果を出す際に、特定のモデルの意見を重要視・あるいは軽視するというもの。 scikit-learn の VotingClassifier であれば weights というオプションでモデルの重みを指定できる。

次のサンプルコードでは、ランダムフォレストとロジスティック回帰の意見を重要視するように重みをつけてみた。

In [2]:
from collections import defaultdict

import numpy as np
from tqdm import tqdm
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB


def main():
    dataset = datasets.load_breast_cancer()
    X, y = dataset.data, dataset.target

    estimators = [
        ('svm', SVC(gamma='scale', probability=True)),
        ('rf', RandomForestClassifier(n_estimators=100)),
        ('logit', LogisticRegression(solver='lbfgs', max_iter=10000)),
        ('knn', KNeighborsClassifier()),
        ('nb', GaussianNB()),
    ]

    accs = defaultdict(list)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, test_index in tqdm(list(skf.split(X, y))):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # 分類器に重みをつける
        voting = VotingClassifier(estimators,
                                  weights=[1, 2, 2, 1, 1])
        voting.fit(X_train, y_train)

        y_pred = voting.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accs['voting'].append(acc)

        for name, estimator in voting.named_estimators_.items():
            y_pred = estimator.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            accs[name].append(acc)

    for name, acc_list in accs.items():
        mean_acc = np.array(acc_list).mean()
        print(name, ':', mean_acc)


if __name__ == '__main__':
    main()

100%|██████████| 5/5 [00:07<00:00,  1.53s/it]

voting : 0.9560937742586555
svm : 0.9138953578636858
rf : 0.9560782487191428
logit : 0.9543393882937432
knn : 0.9350100916006833
nb : 0.9385343890700202


Seed Averaging


---


先ほどの例では、モデルに重み付けしてみたものの結局ランダムフォレストを単体で使った方が性能が良かった。 とはいえ Voting は一つのアルゴリズムだけを使う場合にも性能向上につなげる応用がある。 それが、続いて紹介する Seed Averaging という手法。 これは、同じアルゴリズムでも学習に用いるシード値を異なるものにしたモデルを複数用意して Voting するというやり方。

次のサンプルコードでは、Voting で使うアルゴリズムはランダムフォレストだけになっている。 ただし、初期化するときのシード値がそれぞれ異なっている。

In [3]:
from collections import defaultdict

import numpy as np
from tqdm import tqdm
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold


def main():
    dataset = datasets.load_breast_cancer()
    X, y = dataset.data, dataset.target

    # Seed Averaging
    estimators = [
        ('rf1', RandomForestClassifier(n_estimators=100, random_state=0)),
        ('rf2', RandomForestClassifier(n_estimators=100, random_state=1)),
        ('rf3', RandomForestClassifier(n_estimators=100, random_state=2)),
        ('rf4', RandomForestClassifier(n_estimators=100, random_state=3)),
        ('rf5', RandomForestClassifier(n_estimators=100, random_state=4)),
    ]

    accs = defaultdict(list)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, test_index in tqdm(list(skf.split(X, y))):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        voting = VotingClassifier(estimators)
        voting.fit(X_train, y_train)

        y_pred = voting.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accs['voting'].append(acc)

        for name, estimator in voting.named_estimators_.items():
            y_pred = estimator.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            accs[name].append(acc)

    for name, acc_list in accs.items():
        mean_acc = np.array(acc_list).mean()
        print(name, ':', mean_acc)


if __name__ == '__main__':
    main()  

100%|██████████| 5/5 [00:07<00:00,  1.46s/it]

voting : 0.956078248719143
rf1 : 0.9543238627542306
rf2 : 0.9543083372147182
rf3 : 0.9560782487191428
rf4 : 0.9578326346840551
rf5 : 0.9578326346840551


今回は、最も性能の良い三番目のモデルよりも、わずかながら Voting した結果の方が性能が良くなっている。 これは、各モデルの推論結果を平均することで、最終的なモデルの識別境界がなめらかになる作用が期待できるためと考えられる。

Soft Voting と Hard Voting


---


Voting と一口に言っても、推論結果の出し方には Soft Voting と Hard Voting という二つのやり方がある。 分かりやすいのは Hard Voting で、これは単純に各モデルの意見を多数決で決めるというもの。 もうひとつの Soft Voting は、それぞれのモデルの出した推論結果の確率を平均するというもの。 そこで、続いては、それぞれの手法について詳しく見ていくことにする。